<a href="https://colab.research.google.com/github/Investigator13th/CS246/blob/main/CS246_Colab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 5
## PageRank

In [1]:
from IPython.display import Image
print("Colab 5 Mascot")
Image(url='https://media.giphy.com/media/cCOVfFwDI3awdse5A3/giphy.gif',width=150)

Colab 5 Mascot


### Setup

First of all, we authenticate a Google Drive client to download the dataset we will be processing in this Colab.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1EoolSK32_U74I4FeLox88iuUB_SUUYsI'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('web-Stanford.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading

For this Colab we will be using [NetworkX](https://networkx.github.io), a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks ([tutorial](https://networkx.org/documentation/stable/tutorial.html)).

The dataset we will analyze is a snapshot of the Web Graph centered around [stanford.edu](https://stanford.edu), collected in 2002. Nodes represent pages from Stanford University (stanford.edu) and directed edges represent hyperlinks between them. [[More Info]](http://snap.stanford.edu/data/web-Stanford.html)

In [5]:
import networkx as nx

G = nx.read_edgelist('web-Stanford.txt', create_using=nx.DiGraph)

#### Function to print graph information

In [6]:
def print_graph_info(G, directed=True):
  print("Number of nodes:", len(G.nodes))
  print("Number of edges:", len(G.edges))
  if directed:
    print("Average in-degree:", sum(dict(G.in_degree).values()) / len(G.nodes))
    print("Average out-degree:", sum(dict(G.out_degree).values()) / len(G.nodes))
  else:
    print("Average degree:", sum(dict(G.degree).values()) / len(G.nodes))

In [7]:
print_graph_info(G, True)

Number of nodes: 281903
Number of edges: 2312497
Average in-degree: 8.203165627893283
Average out-degree: 8.203165627893283


### Your Task

To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.

Use NetworkX to identify the **largest** weakly connected component in the ```G``` graph.  **From now on, use this connected component for all the following tasks.**

Print its information.

In [8]:
''' 3 lines of code in total expected. Execution time should be less than 30 seconds'''
components = nx.weakly_connected_components(G)
largest_component = max(components, key=len)
largest_component_G = G.subgraph(largest_component)
print_graph_info(largest_component_G, True)

Number of nodes: 255265
Number of edges: 2234572
Average in-degree: 8.753930229369479
Average out-degree: 8.753930229369479


Compute the PageRank vector, using the default parameters in NetworkX: [https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pageranky](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

In [9]:
''' 1 line of code in total expected. Execution time should be around or less than 15 seconds.'''

# YOUR CODE HERE
pagerank_vector = nx.pagerank(largest_component_G)

In 1999, Barabási and Albert proposed an elegant mathematical model which can generate graphs with topological properties similar to the Web Graph (also called Scale-free Networks).

If you complete the steps below, you should obtain some empirical evidence that the Random Graph model is inferior compared to the Barabási–Albert model when it comes to generating a graph resembling the World Wide Web!

As such, we will use two different graph generator methods, and then we will test how well they approximate the Web Graph structure by means of comparing the respective PageRank vectors. [[NetworkX Graph generators]](https://networkx.github.io/documentation/stable/reference/generators.html#)

Using the parameters ```seed = 1``` and ```directed=False``` where applicable, generate:


1.   a random graph (with the fast method), setting ```n``` equal to the number of nodes in the original connected component, and ```p = 0.00008```
2.   a Barabasi-Albert graph (with the standard method), setting ```n``` equal to the number of nodes in the original connected component, and finding the right ***integer*** value for ```m``` such as the resulting number of edges **approximates by excess(the value that is closest to and satisfies $\geq$)** the number of edges in the original connected component (the largest weakly connected component you attained above)

and compute the PageRank vectors for both graphs. Print generated graph's information, if needed.


In [11]:
''' 6-8 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.
The execution time for one pagerank computation should be around 20 seconds.'''

# YOUR CODE HERE
import networkx as nx
import math
n = largest_component_G.number_of_nodes()
m_edges = largest_component_G.number_of_edges()

seed = 1
p = 0.00008
random_graph = nx.fast_gnp_random_graph(n=n, p=p, seed=seed, directed=False)

# 生成 Barabási-Albert 图
# 首先，计算 m 的值，使得生成的边数略大于原始图的边数
m = int(math.ceil(m_edges / n))
barabasi_albert_graph = nx.barabasi_albert_graph(n=n, m=m, seed=seed)

# 计算两个新图的 PageRank 向量
random_pagerank = nx.pagerank(random_graph)
barabasi_pagerank = nx.pagerank(barabasi_albert_graph)

Compare the PageRank vectors obtained on the generated graphs with the PageRank vector you computed on the original connected component (the largest weakly connected component you attained above).
**Sort** the components of each vector by value, and use cosine similarity as similarity measure.

Feel free to use any implementation of the cosine similarity available in third-party libraries, or implement your own with ```numpy```.

In [22]:
''' 8-10 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''

# YOUR CODE HERE
from scipy.spatial.distance import cosine
def pagerank_to_sort_vector(pagerank_dict):
    # sorted_items = sorted(pagerank_dict.items())
    vector = np.array([item[1] for item in pagerank_dict.items()])
    sorted_vector = np.sort(vector)
    return sorted_vector
    # return np.array([item[1] for item in sorted_items])

def cosine_similarity(vec1, vec2):  ## cosine计算的是余弦距离
    return 1 - cosine(vec1, vec2)

In [23]:
original_pagerank_vector = pagerank_to_sort_vector(pagerank_vector)
random_pagerank_vector = pagerank_to_sort_vector(random_pagerank)
barabasi_pagerank_vector = pagerank_to_sort_vector(barabasi_pagerank)

similarity_original_random = cosine_similarity(original_pagerank_vector, random_pagerank_vector)

similarity_original_barabasi = cosine_similarity(original_pagerank_vector, barabasi_pagerank_vector)

# 输出相似性
print(f"Similarity between original and random graph: {similarity_original_random}")
print(f"Similarity between original and Barabási-Albert graph: {similarity_original_barabasi}")


Similarity between original and random graph: 0.103955647039644
Similarity between original and Barabási-Albert graph: 0.6488673660967236


Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!